In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from collections import defaultdict

sns.set_style("whitegrid")

In [48]:
data=pd.read_csv("/content/training_data_by_patients_and_months.csv")

In [49]:
#(Can be imported from other notebooks later)

from sklearn.model_selection import KFold, train_test_split

class GroupedTemporalHoldoutSplitter:
    """
    Custom scikit-learn compatible splitter for train/test and cross-validation
    based on a temporal holdout strategy for a subset of groups (patients).

    - For a single split (n_splits=1):
        A `test_patient_ratio` of patients are designated as "temporal holdout" patients.
        For these patients, data at or before `cutoff_month` is training data,
        and data after `cutoff_month` is test data.
        All data from other patients go entirely into the training set.

    - For cross-validation (n_splits > 1):
        In each fold, a distinct subset of patients (approximately 1/n_splits of total patients)
        serves as the "temporal holdout" group.
        For these holdout patients, their data at or before `cutoff_month` is added to the
        training set for that fold, and their data after `cutoff_month` forms the
        validation set for that fold.
        All data from patients not in the current holdout group go entirely into the
        training set for that fold.
    """
    def __init__(self, cutoff_month, n_splits=5, test_patient_ratio=0.2,
                 random_state=None, shuffle_patients=True):
        """
        Args:
            cutoff_month (int): The visit_month used to split the data for
                                temporal holdout patients.
            n_splits (int): Number of splitting iterations in the cross-validator.
                            If 1, performs a single train/test split.
            test_patient_ratio (float): Proportion of patients to designate as
                                        temporal holdout in a single split (n_splits=1).
                                        Ignored if n_splits > 1.
            random_state (int, optional): Seed for shuffling patient IDs.
                                          Ensures reproducibility.
            shuffle_patients (bool): Whether to shuffle patient IDs before splitting.
                                     Default is True.
        """
        if not isinstance(n_splits, int) or n_splits <= 0:
            raise ValueError("n_splits must be a positive integer.")
        if not isinstance(cutoff_month, int) or cutoff_month < 0:
            raise ValueError("cutoff_month must be a non-negative integer.")
        if n_splits == 1 and not (0 < test_patient_ratio < 1):
            raise ValueError("test_patient_ratio must be between 0 and 1 (exclusive) for a single split.")

        self.cutoff_month = cutoff_month
        self.n_splits = n_splits
        self.test_patient_ratio = test_patient_ratio
        self.random_state = random_state
        self.shuffle_patients = shuffle_patients

    def get_n_splits(self, X=None, y=None):
        """Returns the number of splitting iterations in the cross-validator."""
        return self.n_splits

    def split(self, X, y=None, groups=None):
        """
        Generate indices to split data into training and test set.

        Args:
            X (pd.DataFrame): DataFrame containing the data. Must include a
                              'visit_month' column.
            y (array-like, optional): The target variable. Not used in this
                                      splitting logic directly.
            groups (pd.Series): Series containing the group labels (e.g., patient IDs),
                                aligned with the index of X.

        Yields:
            train_idx (np.array): The training set indices (positional) for that split.
            test_idx (np.array): The testing set indices (positional) for that split.
        """
        if groups is None:
            raise ValueError("The 'groups' parameter cannot be None and should contain patient IDs.")
        if not isinstance(X, pd.DataFrame) or 'visit_month' not in X.columns:
            raise ValueError("X must be a pandas DataFrame with a 'visit_month' column.")
        if not isinstance(groups, pd.Series) or not X.index.equals(groups.index):
            raise ValueError("X.index and groups.index must be aligned.")

        unique_patient_ids = groups.unique()

        original_indices_map = {original_idx: pos_idx for pos_idx, original_idx in enumerate(X.index)}

        if self.n_splits == 1:
            # --- Single Train/Test Split ---
            if len(unique_patient_ids) < 2:
                 raise ValueError(f"Not enough unique patients ({len(unique_patient_ids)}) to perform a train/test split "
                                  f"with test_patient_ratio={self.test_patient_ratio}.")

            # Split unique patient IDs into 'full_train_patients' and 'temporal_holdout_patients'
            # The 'test_size' in train_test_split refers to the proportion for the second returned array.
            full_train_patient_ids, temporal_holdout_patient_ids = train_test_split(
                unique_patient_ids,
                test_size=self.test_patient_ratio, # This proportion becomes temporal_holdout_patients
                random_state=self.random_state,
                shuffle=self.shuffle_patients
            )

            # Handle cases where one group might be empty due to small numbers and rounding
            if len(temporal_holdout_patient_ids) == 0 and len(full_train_patient_ids) > 0:
                temporal_holdout_patient_ids = np.array([full_train_patient_ids[-1]])
                full_train_patient_ids = full_train_patient_ids[:-1]
            if len(full_train_patient_ids) == 0 and len(temporal_holdout_patient_ids) > 0:
                 full_train_patient_ids = np.array([temporal_holdout_patient_ids[-1]])
                 temporal_holdout_patient_ids = temporal_holdout_patient_ids[:-1]
            if len(temporal_holdout_patient_ids) == 0 or len(full_train_patient_ids) == 0:
                 raise ValueError("Could not create a valid split. One of the patient groups is empty.")

            train_mask = pd.Series(False, index=X.index)
            test_mask = pd.Series(False, index=X.index)
            train_mask[groups.isin(full_train_patient_ids)] = True
            is_temporal_holdout_mask = groups.isin(temporal_holdout_patient_ids)
            train_mask[is_temporal_holdout_mask & (X['visit_month'] <= self.cutoff_month)] = True
            test_mask[is_temporal_holdout_mask & (X['visit_month'] > self.cutoff_month)] = True

            train_indices_pos = np.array([original_indices_map[idx] for idx in X.index[train_mask]])
            test_indices_pos = np.array([original_indices_map[idx] for idx in X.index[test_mask]])

            if len(test_indices_pos) == 0:
                raise ValueError("Test set is empty for the single split. "
                                 "Ensure temporal holdout patients have data after the cutoff_month.")
            if len(train_indices_pos) == 0:
                 raise ValueError("Train set is empty for the single split. This is unexpected.")

            yield train_indices_pos, test_indices_pos

        else:
            # --- Cross-Validation Split ---
            if len(unique_patient_ids) < self.n_splits:
                raise ValueError(f"Number of unique patients ({len(unique_patient_ids)}) "
                                 f"is less than n_splits ({self.n_splits}). "
                                 "Reduce n_splits or provide more diverse patient data.")

            patient_folder = KFold(n_splits=self.n_splits, shuffle=self.shuffle_patients,
                                   random_state=self.random_state)

            folds_generated = 0
            for _, val_patient_indices_in_unique_list in patient_folder.split(unique_patient_ids):
                # Patients designated as temporal holdout for this CV fold
                cv_temporal_holdout_ids = unique_patient_ids[val_patient_indices_in_unique_list]

                cv_train_mask = pd.Series(False, index=X.index)
                cv_val_mask = pd.Series(False, index=X.index)

                # Patients NOT in cv_temporal_holdout_ids have all their data in training for this fold
                is_full_train_for_cv_fold_mask = ~groups.isin(cv_temporal_holdout_ids)
                cv_train_mask[is_full_train_for_cv_fold_mask] = True
                is_cv_temporal_holdout_mask = groups.isin(cv_temporal_holdout_ids)
                cv_train_mask[is_cv_temporal_holdout_mask & (X['visit_month'] <= self.cutoff_month)] = True
                cv_val_mask[is_cv_temporal_holdout_mask & (X['visit_month'] > self.cutoff_month)] = True

                train_indices_pos = np.array([original_indices_map[idx] for idx in X.index[cv_train_mask]])
                val_indices_pos = np.array([original_indices_map[idx] for idx in X.index[cv_val_mask]])

                if len(val_indices_pos) == 0:
                    print(f"Warning: CV Fold - Validation set is empty for temporal_holdout_ids: {list(cv_temporal_holdout_ids)}. Skipping this fold.")
                    continue
                if len(train_indices_pos) == 0:
                    print(f"Warning: CV Fold - Training set is empty. Skipping this fold.")
                    continue

                yield train_indices_pos, val_indices_pos
                folds_generated += 1

            if folds_generated == 0 and self.n_splits > 0 :
                 raise ValueError("No valid CV splits were generated. Check data and cutoff_month. "
                                  "It's possible no temporal holdout patients had data after the cutoff month in any fold configuration.")


In [50]:
CUTOFF_MONTH = 24

# For cross-validation
cv_splitter = GroupedTemporalHoldoutSplitter(
    cutoff_month=CUTOFF_MONTH,
    n_splits=5,
    random_state=42
)

In [51]:
data.columns

Index(['visit_id', 'AADDTWEPFASGK',
       'AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K',
       'AAFTEC(UniMod_4)C(UniMod_4)QAADK', 'AANEVSSADVK',
       'AATGEC(UniMod_4)TATVGKR', 'AATVGSLAGQPLQER', 'AAVYHHFISDGVR',
       'ADDKETC(UniMod_4)FAEEGK', 'ADDKETC(UniMod_4)FAEEGKK',
       ...
       'Q9UNU6', 'Q9Y646', 'Q9Y6R7', 'patient_id', 'visit_month', 'updrs_1',
       'updrs_2', 'updrs_3', 'updrs_4', 'upd23b_clinical_state_on_medication'],
      dtype='object', length=1203)

In [52]:
updrs_cols = ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']

mae_results = {}
mae_results['LOCF'] = defaultdict(list)

for i,(training_index, test_index) in enumerate(cv_splitter.split(data,groups=data["patient_id"])):
    train, holdout=data.iloc[training_index], data.iloc[test_index]


    # --- 1. Last Observation Carried Forward (LOCF) from Cutoff ---
    print("\n--- Last Observation Carried Forward (LOCF) ---")
    holdout_locf = holdout.copy()

    for patient_id in holdout_locf['patient_id'].unique():
        # Get the patient's training data at or before the cutoff
        patient_train_data_at_cutoff = train[
            (train['patient_id'] == patient_id) &
            (train['visit_month'] <= CUTOFF_MONTH)
        ]

        if not patient_train_data_at_cutoff.empty:
            # Find the latest visit at or before the cutoff
            last_visit_before_cutoff = patient_train_data_at_cutoff.loc[
                patient_train_data_at_cutoff['visit_month'].idxmax()
            ]
            for col in updrs_cols:
              holdout_locf.loc[holdout_locf['patient_id'] == patient_id, f'{col}_pred_locf'] = last_visit_before_cutoff[col]
        else:
            # Handle case where patient might not have data exactly at cutoff (should not happen with your split logic)
            # Or if a patient in test somehow wasn't in train (error in split)
            print("Warning: error in split!")


    # Calculate MAE for LOCF predictions
    print("\n--- MAE for LOCF Model ---")

    for col in updrs_cols:
        actual_values = holdout_locf[col]
        predicted_values = holdout_locf[f'{col}_pred_locf']

        # Ensure no NaNs if a patient in test had no prior data
        valid_indices = predicted_values.notna() & actual_values.notna()
        if valid_indices.sum() > 0: # Only calculate if there are valid predictions
            mae = mean_absolute_error(actual_values[valid_indices], predicted_values[valid_indices])
            mae_results['LOCF'][col].append(mae)
            print(f"MAE for {col} (LOCF): {mae:.4f}")
        else:
            print(f"MAE for {col} (LOCF): Not enough valid predictions to calculate.")
            mae_results['LOCF'][col].append(np.nan)



--- Last Observation Carried Forward (LOCF) ---

--- MAE for LOCF Model ---
MAE for updrs_1 (LOCF): 3.0607
MAE for updrs_2 (LOCF): 3.0393
MAE for updrs_3 (LOCF): 7.0919
MAE for updrs_4 (LOCF): 1.9375

--- Last Observation Carried Forward (LOCF) ---

--- MAE for LOCF Model ---
MAE for updrs_1 (LOCF): 3.4480
MAE for updrs_2 (LOCF): 3.1448
MAE for updrs_3 (LOCF): 6.4128
MAE for updrs_4 (LOCF): 1.1277

--- Last Observation Carried Forward (LOCF) ---

--- MAE for LOCF Model ---
MAE for updrs_1 (LOCF): 2.4276
MAE for updrs_2 (LOCF): 1.9345
MAE for updrs_3 (LOCF): 7.2975
MAE for updrs_4 (LOCF): 1.3299

--- Last Observation Carried Forward (LOCF) ---

--- MAE for LOCF Model ---
MAE for updrs_1 (LOCF): 3.1096
MAE for updrs_2 (LOCF): 2.6941
MAE for updrs_3 (LOCF): 6.2917
MAE for updrs_4 (LOCF): 1.8705


One problem here is that all the patients who has some data in the test set will be ignored in cross validation.

In [53]:
# historical mean
mae_results['PatientMean'] = defaultdict(list)
for i,(training_index, test_index) in enumerate(cv_splitter.split(data,groups=data["patient_id"])):
  train, holdout=data.iloc[training_index], data.iloc[test_index]
  holdout_mean = holdout.copy()

  for patient_id in holdout_mean['patient_id'].unique():
      patient_train_data_before_cutoff = train[
          (train['patient_id'] == patient_id) &
          (train['visit_month'] <= CUTOFF_MONTH)
      ]
      if not patient_train_data_before_cutoff.empty:
        for col in updrs_cols:
            mean_val = patient_train_data_before_cutoff[col].mean()
            holdout_mean.loc[holdout_mean['patient_id'] == patient_id, f'{col}_pred_mean'] = mean_val
      else:
        print("Warning: error in split!")

  print("\n--- MAE for Patient Historical Mean Model ---")

  for col in updrs_cols:
      actual_values = holdout_mean[col]
      predicted_values = holdout_mean[f'{col}_pred_mean']

      valid_indices = predicted_values.notna() & actual_values.notna()
      if valid_indices.sum() > 0:
          mae = mean_absolute_error(actual_values[valid_indices], predicted_values[valid_indices])
          mae_results['PatientMean'][col].append(mae)
          print(f"MAE for {col} (PatientMean): {mae:.4f}")
      else:
          print(f"MAE for {col} (PatientMean): Not enough valid predictions to calculate.")
          mae_results['PatientMean'][col].append(np.nan)

print("the mean absolute errors for LOCF are", mae_results['LOCF'])

print("the mean absolute errors for historical mean are", mae_results['PatientMean'])


--- MAE for Patient Historical Mean Model ---
MAE for updrs_1 (PatientMean): 2.9469
MAE for updrs_2 (PatientMean): 3.1733
MAE for updrs_3 (PatientMean): 7.6486
MAE for updrs_4 (PatientMean): 1.9309

--- MAE for Patient Historical Mean Model ---
MAE for updrs_1 (PatientMean): 3.3363
MAE for updrs_2 (PatientMean): 3.2412
MAE for updrs_3 (PatientMean): 7.2167
MAE for updrs_4 (PatientMean): 1.2728

--- MAE for Patient Historical Mean Model ---
MAE for updrs_1 (PatientMean): 2.4256
MAE for updrs_2 (PatientMean): 2.0217
MAE for updrs_3 (PatientMean): 7.3438
MAE for updrs_4 (PatientMean): 1.4130

--- MAE for Patient Historical Mean Model ---
MAE for updrs_1 (PatientMean): 3.5193
MAE for updrs_2 (PatientMean): 2.9578
MAE for updrs_3 (PatientMean): 6.3966
MAE for updrs_4 (PatientMean): 1.5692
the mean absolute errors for LOCF are defaultdict(<class 'list'>, {'updrs_1': [3.0607142857142855, 3.4479638009049776, 2.4275862068965517, 3.1095890410958904], 'updrs_2': [3.039285714285714, 3.14479638009

In [54]:
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings
warnings.filterwarnings("ignore") # Suppress convergence warnings, etc. for brevity

In [59]:
mae_results['AR'] = defaultdict(list)
mae_results['SARIMA'] = defaultdict(list)
mae_results['ES'] = defaultdict(list)
for j,(training_index, test_index) in enumerate(cv_splitter.split(data,groups=data["patient_id"])):
  train, holdout=data.iloc[training_index], data.iloc[test_index]
  for target_updrs_col in updrs_cols:

    print(f"\n--- Classical Time Series Models for {target_updrs_col} ---")
    holdout_ts = holdout.copy()
    for patient_id in holdout_ts['patient_id'].unique():
        print(f"\nProcessing Patient ID: {patient_id}")
        # Get patient's training data for this specific UPDRS score
        patient_train_series = train[
            (train['patient_id'] == patient_id) &
            (train['visit_month'] <= CUTOFF_MONTH)
        ].set_index('visit_month')[target_updrs_col].sort_index()

        # Get the visit months for which we need predictions (test set)
        patient_test_visit_months = holdout_ts[holdout_ts['patient_id'] == patient_id]['visit_month'].sort_values()

        if len(patient_train_series) < 3: # Need enough data points to fit these models
            print(f"  Not enough training data for patient {patient_id} (<= {CUTOFF_MONTH} months). Skipping TS models.")
            holdout_ts.loc[holdout_ts['patient_id'] == patient_id, f'{target_updrs_col}_pred_ar'] = np.nan
            holdout_ts.loc[holdout_ts['patient_id'] == patient_id, f'{target_updrs_col}_pred_sarima'] = np.nan
            holdout_ts.loc[holdout_ts['patient_id'] == patient_id, f'{target_updrs_col}_pred_es'] = np.nan
            continue

        # --- Autoregressive (AR) Model ---
        try:
            # Determine lags, e.g., min(len(series)//2 -1, 12) or use information criteria
            lags_ar = min(len(patient_train_series) // 2, 5)

            print(f"lag is {lags_ar}")
            if lags_ar > 0:
                model_ar = AutoReg(patient_train_series, lags=lags_ar, old_names=False)
                res_ar = model_ar.fit()
                print(model_ar.params)
                # Forecast for each future month in the test set
                # AutoReg predict needs start and end *indices* relative to the end of the training series
                start_pred_idx = len(patient_train_series)
                # This is tricky because test visit_months are not necessarily consecutive from train
                # For simplicity, we'll just predict a sequence and try to map.
                # A more robust way would be to re-fit or use a model that handles explicit future indices.
                # Here, we predict steps ahead.
                num_steps_to_predict = len(patient_test_visit_months)
                pred_ar = res_ar.predict(start=start_pred_idx, end=start_pred_idx + num_steps_to_predict -1)
                # This simple mapping assumes test months are somewhat consecutive after train
                # This part needs careful handling for real irregular test months
                for i, month_to_predict in enumerate(patient_test_visit_months):
                    if i < len(pred_ar):
                        holdout_ts.loc[(holdout_ts['patient_id'] == patient_id) & (holdout_ts['visit_month'] == month_to_predict), f'{target_updrs_col}_pred_ar'] = pred_ar.iloc[i]
                    else: # pragma: no cover
                        holdout_ts.loc[(holdout_ts['patient_id'] == patient_id) & (holdout_ts['visit_month'] == month_to_predict), f'{target_updrs_col}_pred_ar'] = np.nan # if not enough predictions
            else:
                holdout_ts.loc[holdout_ts['patient_id'] == patient_id, f'{target_updrs_col}_pred_ar'] = np.nan
        except Exception as e:
            print(f"  AR model failed for patient {patient_id}: {e}")
            holdout_ts.loc[holdout_ts['patient_id'] == patient_id, f'{target_updrs_col}_pred_ar'] = np.nan

        # --- SARIMA Model (example, parameters p,d,q,P,D,Q,s need tuning per series) ---
        try:
            start_pred_idx = len(patient_train_series)
            num_steps_to_predict = len(patient_test_visit_months)
            # Example: ARIMA(1,1,1) - needs tuning
            # Seasonality 's' is tricky here. If visits are every 6 months, s=2 (for 2 periods in a year)
            # but patient series are short. Let's try a non-seasonal ARIMA for simplicity.
            model_sarima = SARIMAX(patient_train_series, order=(1,1,1), seasonal_order=(0,0,0,0))
            res_sarima = model_sarima.fit(disp=False)
            # Similar prediction logic challenge as AR for specific future months
            pred_sarima = res_sarima.predict(start=start_pred_idx, end=start_pred_idx + num_steps_to_predict -1)
            for i, month_to_predict in enumerate(patient_test_visit_months):
                if i < len(pred_sarima):
                    holdout_ts.loc[(holdout_ts['patient_id'] == patient_id) & (holdout_ts['visit_month'] == month_to_predict), f'{target_updrs_col}_pred_sarima'] = pred_sarima.iloc[i]
                else: # pragma: no cover
                    holdout_ts.loc[(holdout_ts['patient_id'] == patient_id) & (holdout_ts['visit_month'] == month_to_predict), f'{target_updrs_col}_pred_sarima'] = np.nan
        except Exception as e:
            print(f"  SARIMA model failed for patient {patient_id}: {e}")
            holdout_ts.loc[holdout_ts['patient_id'] == patient_id, f'{target_updrs_col}_pred_sarima'] = np.nan




        # --- Exponential Smoothing (Holt's Linear Trend) ---
        try:
            start_pred_idx = len(patient_train_series)
            num_steps_to_predict = len(patient_test_visit_months)
            model_es = ExponentialSmoothing(patient_train_series, trend='add', seasonal=None)
            res_es = model_es.fit()
            pred_es = res_es.forecast(steps=num_steps_to_predict)
            for i, month_to_predict in enumerate(patient_test_visit_months):
                if i < len(pred_es):
                    holdout_ts.loc[(holdout_ts['patient_id'] == patient_id) & (holdout_ts['visit_month'] == month_to_predict), f'{target_updrs_col}_pred_es'] = pred_es.iloc[i]
                else: # pragma: no cover
                    holdout_ts.loc[(holdout_ts['patient_id'] == patient_id) & (holdout_ts['visit_month'] == month_to_predict), f'{target_updrs_col}_pred_es'] = np.nan
        except Exception as e:
            print(f"  Exponential Smoothing model failed for patient {patient_id}: {e}")
            holdout_ts.loc[holdout_ts['patient_id'] == patient_id, f'{target_updrs_col}_pred_es'] = np.nan


    actual_values = holdout_ts[col]
    predicted_values_ar = holdout_ts[f'{target_updrs_col}_pred_ar']
    predicted_values_sarima = holdout_ts[f'{target_updrs_col}_pred_sarima']
    predicted_values_es = holdout_ts[f'{target_updrs_col}_pred_es']

    valid_indices_ar = predicted_values_ar.notna() & actual_values.notna()
    valid_indices_sarima = predicted_values_sarima.notna() & actual_values.notna()
    valid_indices_es = predicted_values_es.notna() & actual_values.notna()
    print(f"there are {predicted_values_ar.isna().sum()} missing values for autoregressive model")
    print(f"there are {predicted_values_sarima.isna().sum()} missing values for SARIMA model")
    print(f"there are {predicted_values_es.isna().sum()} missing values for Exponential Smoothing model")
    if valid_indices_ar.sum() > 0:
        mae_ar= mean_absolute_error(actual_values[valid_indices_ar], predicted_values_ar[valid_indices_ar])
        mae_results['AR'][target_updrs_col].append(mae_ar)
        print(f"MAE for {target_updrs_col} (AutoRegressive): {mae_ar:.4f}")
    else:
        print(f"MAE for {target_updrs_col} (AutoRegressive): Not enough valid predictions to calculate.")
        mae_results['AR'][target_updrs_col].append(np.nan)
    if valid_indices_sarima.sum() > 0:
      mae_sarima= mean_absolute_error(actual_values[valid_indices_sarima], predicted_values_sarima[valid_indices_sarima])
      print(f"MAE for {target_updrs_col} (SARIMA): {mae_sarima:.4f}")
      mae_results['SARIMA'][target_updrs_col].append(mae_sarima)
    else:
      print(f"MAE for {target_updrs_col} (SARIMA): Not enough valid predictions to calculate.")
      mae_results['SARIMA'][target_updrs_col].append(np.nan)
    if valid_indices_es.sum() > 0:
      mae_es= mean_absolute_error(actual_values[valid_indices_es], predicted_values_es[valid_indices_es])
      print(f"MAE for {target_updrs_col} (Exponential Smoothing): {mae_es:.4f}")
      mae_results['ES'][target_updrs_col].append(mae_es)
    else:
      print(f"MAE for {target_updrs_col} (Exponential Smoothing): Not enough valid predictions to calculate.")
      mae_results['ES'][target_updrs_col].append(np.nan)





--- Classical Time Series Models for updrs_1 ---

Processing Patient ID: 10718.0
lag is 2
  AR model failed for patient 10718.0: division by zero

Processing Patient ID: 12755.0
lag is 3
  AR model failed for patient 12755.0: division by zero

Processing Patient ID: 14242.0
lag is 2
  AR model failed for patient 14242.0: division by zero

Processing Patient ID: 1517.0
lag is 2
  AR model failed for patient 1517.0: division by zero

Processing Patient ID: 15245.0
lag is 3
  AR model failed for patient 15245.0: division by zero

Processing Patient ID: 15590.0
lag is 3
  AR model failed for patient 15590.0: division by zero

Processing Patient ID: 16574.0
lag is 2
  AR model failed for patient 16574.0: division by zero

Processing Patient ID: 18183.0
lag is 3
  AR model failed for patient 18183.0: division by zero

Processing Patient ID: 20404.0
lag is 3
  AR model failed for patient 20404.0: division by zero

Processing Patient ID: 20791.0
lag is 1
  AR model failed for patient 20791.0: